this notebook is a first look at mapping a real person to authors on papers

when the final requirement is defined this can be revisited

In [95]:
import pandas as pd
import numpy as np

import psycopg2

import sqlUtils as q

In [96]:
query = """
select *
from 
    match_prob
where
    prob > 0.9
"""

df = q.query(query, 'sqlConfig.json')

In [97]:
df['apid1'] = df.asid1.astype(str) + '_' + df.pid1.astype(str)
df['apid2'] = df.asid2.astype(str) + '_' + df.pid2.astype(str)

In [98]:
df.head()

,asid1,pid1,asid2,pid2,prob,apid1,apid2
0,787998,538833,787998,376653,0.976562,787998_538833,787998_376653
1,390550,538832,390550,128175,0.976562,390550_538832,390550_128175
2,390550,538832,488447,166886,0.976562,390550_538832,488447_166886
3,356169,538832,356169,198571,0.976562,356169_538832,356169_198571
4,356169,538832,356169,198630,0.976562,356169_538832,356169_198630


In [78]:
df = pd.DataFrame(data={'asid1':[1, 1, 2, 3, 4, 5, 5], 'pid1':[1, 1, 2, 3, 4, 5, 5], 'asid2':[2, 3, 3, 2, 5, 6, 1], 'pid2':[2, 3, 3, 2, 5, 6, 1]})
df['apid1'] = df.asid1.astype(str) + '_' + df.pid1.astype(str)
df['apid2'] = df.asid2.astype(str) + '_' + df.pid2.astype(str)
df

,asid1,asid2,pid1,pid2,apid1,apid2
0,1,2,1,2,1_1,2_2
1,1,3,1,3,1_1,3_3
2,2,3,2,3,2_2,3_3
3,3,2,3,2,3_3,2_2
4,4,5,4,5,4_4,5_5
5,5,6,5,6,5_5,6_6
6,5,1,5,1,5_5,1_1


In [117]:
def update_author_df(author, df):
    
    i = 0
    
    if len(author.index) == 0:
    
        index = 0
        author = author.append({'aid': 1, 'apid': df.at[index, 'apid1'], 'asid': df.at[index, 'asid1'], 'pid': df.at[index, 'pid1']}, ignore_index=True)
        author = author.append({'aid': 1, 'apid': df.at[index, 'apid2'], 'asid': df.at[index, 'asid2'], 'pid': df.at[index, 'pid2']}, ignore_index=True)
        author.set_index('apid', inplace=True)
        i = 1

    for index in df.index[i:]:       

        ########

        apid1 = df.at[index, 'apid1']
        apid2 = df.at[index, 'apid2']

        ## look for an inconsitency where both apid are in author pointing to two different aid
        ## if so update
        try: 
            aid1 = author.at[apid1, 'aid']
            aid2 = author.at[apid2, 'aid']

            if aid1 != aid2:
                author.loc[author.aid == aid2, 'aid'] = aid1
        except:
            pass        
        
        ## if apid1 not in author create new aid and add new row
        if apid1 not in author.index:

            aid1 = author.aid.max() + 1

            asid1 = df.at[index, 'asid1']
            pid1  = df.at[index,  'pid1']        

            author = author.append(pd.DataFrame(index=[apid1], data={'aid':[aid1], 'asid':[asid1], 'pid':[pid1]}))

        #########


        ## if apid2 not in author add new row with apid1's aid
        if apid2 not in author.index:
           
            aid2 = author.loc[apid1, 'aid']

            asid2 = df.at[index, 'asid2']
            pid2  = df.at[index,  'pid2']

            author = author.append(pd.DataFrame(index=[apid2], data={'aid':[aid2], 'asid':[asid2], 'pid':[pid2]}))
    
    return author


author = pd.DataFrame(columns=['aid', 'apid', 'asid', 'pid'], dtype=np.int32)
author = update_author_df(author, df[:-1])
    
    ### what if there is a clash?
    
author.aid.unique()

array([  1, 102, 103, 105,   5, 134, 135,   8,   9,  10,  11,  12, 123,
       124,  15,  16,  17, 115, 116, 117, 118, 104,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34, 128, 129, 132,  69,  39,
        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56, 130,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  91,  71,  97,  73,  74,  75,  76,  95,  78,  79,  80,  81,
        82,  83,  84,  85,  86,  87,  88,  89,  90,  92,  98,  99, 100,
       101, 106, 107, 108, 109, 110, 111, 112, 113, 114, 119, 120, 121,
       122, 125, 126, 127, 131, 133, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150], dtype=int64)

In [111]:
author = pd.DataFrame(columns=['aid', 'apid', 'asid', 'pid'], dtype=np.int32)

In [104]:
%timeit update_author_df(author, df)

1.72 s ± 28.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [112]:
author = pd.DataFrame(columns=['aid', 'apid', 'asid', 'pid'], dtype=np.int32)

In [106]:
%timeit update_author_df(author, df)

1.69 s ± 25.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
len(df)

1647

In [113]:
author.head(10)

,aid,apid,asid,pid
